In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import font

class SudokuSolver:
    def __init__(self):
        self.board = [[0]*9 for _ in range(9)]

    def set_board(self, puzzle):
        for i in range(9):
            for j in range(9):
                self.board[i][j] = puzzle[i][j]

    def solve(self):
        empty_cell = self.find_empty()
        if not empty_cell:
            return True  # Puzzle solved
        row, col = empty_cell
        for num in range(1, 10):
            if self.is_valid(num, (row, col)):
                self.board[row][col] = num
                if self.solve():
                    return True
                self.board[row][col] = 0
        return False

    def find_empty(self):
        for i in range(9):
            for j in range(9):
                if self.board[i][j] == 0:
                    return (i, j)
        return None

    def is_valid(self, num, pos):
        row, col = pos
        for i in range(9):
            if self.board[row][i] == num or self.board[i][col] == num:
                return False
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(start_row, start_row + 3):
            for j in range(start_col, start_col + 3):
                if self.board[i][j] == num:
                    return False
        return True

    def is_board_valid(self):
        for i in range(9):
            for j in range(9):
                if self.board[i][j] != 0:
                    num = self.board[i][j]
                    self.board[i][j] = 0  # Temporarily remove the number for validation
                    if not self.is_valid(num, (i, j)):
                        self.board[i][j] = num  # Restore the number
                        return False
                    self.board[i][j] = num  # Restore the number
        return True

    def solve_sudoku(self):
        if not self.is_board_valid():
            return False  # The initial board configuration is invalid
        return self.solve()


class SudokuBoard:
    def __init__(self, master):
        self.master = master
        self.master.title("Sudoku")

        self.sudoku_solver = SudokuSolver()

        self.create_widgets()

    def create_widgets(self):
        self.input_frame = tk.Frame(self.master)
        self.input_frame.pack(pady=10)

        self.input_label = tk.Label(self.input_frame, text="Enter Sudoku Puzzle:",font=('Arial',16))
        self.input_label.grid(row=0, column=0, columnspan=9)

        self.entries = []
        for i in range(9):
            for j in range(9):
                entry = tk.Entry(self.input_frame, width=3, font=('Arial', 20))
                entry.grid(row=i//3*3 + i%3 + 1, column=j//3*3 + j%3, padx=1, pady=1)
                if j in [2, 5]:
                    entry.grid(padx=(0, 8))
                if i in [2, 5]:
                    entry.grid(pady=(0, 8))
                self.entries.append(entry)

        self.solve_button = tk.Button(self.input_frame, text="Solve", command=self.solve_sudoku, font=('Arial', 16))
        self.solve_button.grid(row=10, column=0, columnspan=4, pady=10)

        self.reset_button = tk.Button(self.input_frame, text="Reset", command=self.reset_puzzle, font=('Arial', 16))
        self.reset_button.grid(row=10, column=5, columnspan=4, pady=10)


        self.output_frame = tk.Frame(self.master)
        self.output_frame.pack(pady=5)

        self.output_label = tk.Label(self.output_frame, text="Solved Sudoku:",font=('Arial',16))
        self.output_label.pack()

        self.output_text = tk.Text(self.output_frame, height=20, width=25, font=('Arial', 16))
        self.output_text.pack()

    def get_input_puzzle(self):
        puzzle = [[0] * 9 for _ in range(9)]

        for i in range(9):
            row = []
            for j in range(9):
                value = self.entries[i * 9 + j].get()
                if value:
                    if not value.isdigit():
                        messagebox.showerror("Error", "Invalid input: number must be a digit!")
                        return None
                    num = int(value)
                    if num < 1 or num > 9:
                        messagebox.showerror("Error", "Invalid input: number must be from 1-9!")
                        return None
                    if not self.sudoku_solver.is_valid(num, (i, j)):
                        messagebox.showerror("Error", "Invalid input: number must not be repeated in row, column, or box!")
                        return None
                    row.append(num)
                else:
                    row.append(0)
            puzzle[i] = row
        return puzzle


        

    def solve_sudoku(self):
        puzzle = self.get_input_puzzle()
        self.sudoku_solver.set_board(puzzle)
        if self.sudoku_solver.solve():
            solved_puzzle = "\n"
            i = 1
            for row in self.sudoku_solver.board:
                solved_puzzle += "     "
                solved_puzzle += "  ".join(map(str, row[:3]))
                for j in range(3, len(row)):
                    if j % 3 == 0: 
                        solved_puzzle += "     " 
                    else:
                        solved_puzzle += "  "
                    solved_puzzle += str(row[j])
                solved_puzzle += "\n"
                if i > 0 and i % 3 == 0:
                    solved_puzzle += "\n"
                i += 1
            self.output_text.delete("1.0", tk.END)
            self.output_text.insert(tk.END, solved_puzzle)
        else:
            messagebox.showerror("Error", "No solution exists for this Sudoku puzzle!")


    def reset_puzzle(self):
        for entry in self.entries:
            entry.delete(0, tk.END)
        self.output_text.delete("1.0", tk.END)
        puzzle = [[0]*9 for _ in range(9)]
        self.sudoku_solver.set_board(puzzle)

def main():
    root = tk.Tk()
    #root.attributes('-fullscreen', True)
    width = root.winfo_screenwidth() 
    height = root.winfo_screenheight()  
    root.geometry("%dx%d+0+0" % (width, height))  
    SudokuBoard(root)
    root.mainloop()


if __name__ == "__main__":
    main()